In [9]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 
import csv
from itertools import islice
import ast

In [2]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

==
Logging in..
Dimcli - Dimensions API Client (v1.2)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.9
Method: manual login


In [4]:
names = {}
#CHANGE THE FILE NAME TO YOUR FILE NAME FROM DimensionsAI_IDs.ipynb
with open('1995_2010.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)  
    for row in reader:
        name = row[0]
        openIds_str = row[2]  
        openIds_list = ast.literal_eval(openIds_str)
        names[name] = openIds_list

print(names)

{"alan d'andrea": ['ur.012727024557.25'], 'james fagin': ['ur.01002446502.38'], 'daniel haber': ['ur.01221020161.61'], 'ethylin jabs': ['ur.0757041626.26'], 'karl nath': ['ur.0621613670.95'], 'bishr omary': ['ur.014730211054.29'], 'john raymond': ['ur.0717265610.95'], 'fuad ziyadeh': ['ur.0761676730.05'], 'maria abreu': ['ur.0606477677.70'], 'michael ackerman': ['ur.01227355747.25'], 'mariana kaplan': ['ur.012647507142.53'], 'feroz papa': ['ur.01147037443.33'], 'paul taylor': ['ur.013436160257.94'], 'loren walensky': ['ur.01324277771.74'], 'yiping yang': ['ur.0707211155.90'], 'steven albelda': ['ur.015000016117.95'], 'richard  anderson': ['ur.010576736217.64'], 'peter arvan': ['ur.01002760245.88'], 'adam asch': ['ur.0615715006.01'], 'john belmont': ['ur.01140262452.40'], 'bradford berk': ['ur.012523312277.63'], 'morris birnbaum': ['ur.01033446252.27'], 'richard blumberg': ['ur.0633257054.96'], 'douglas bradley': ['ur.01153467234.36'], 'kenneth bridges': ['ur.016162551417.02'], 'william

In [47]:
#deleted orange_book
search_Var = "[abstract + additional_filters + application_number + assignee_cities + assignee_countries + assignee_names + assignee_state_codes + assignees + associated_grant_ids + category_bra + category_for + category_for_2020 + category_hra + category_hrcs_hc + category_hrcs_rac + category_icrp_cso + category_icrp_ct + category_rcdc + claims_amount + current_assignee_names + current_assignees + date + date_inserted + dimensions_url + expiration_date + family_count + family_id + federal_support + filing_date + filing_status + funder_countries + funders + granted_date + granted_year + id + inventor_names + inventors + ipcr + jurisdiction + kind + legal_status + original_assignee_names + original_assignees + priority_date + priority_year + publication_date + publication_ids + publication_year + publications + reference_ids + researchers + score + times_cited + title + year]"


In [48]:
#checks output of findby_ID 
def single_id(person_id):
    #[basics+categories+extras]
    res3 = dsl.query_iterative(f"""search patents where researchers = "{person_id}" return patents {search_Var}""") #[basics+categories+extras+family_count+family_id+federal_support+filing_date+granted_date+publication_ids]""")
    listDict3 = res3.json['patents']
    number = res3.json['_stats']['total_count']
    print(f'{person_id} id has {number} patents')
    name_list = []
    for patent in listDict3:
        name_list.append(patent)
        

    return(name_list)


#"ur.0706401556.06", Elliot Androphy
#ur.01112411132.31
single_id("ur.0706401556.06")

Starting iteration with limit=1000 skip=0 ...
0-97 / 97 (0.89s)
===
Records extracted: 97


ur.0706401556.06 id has 97 patents


[{'additional_filters': ['Research Organization'],
  'application_number': 'ZA2018/05486',
  'assignee_cities': [{'id': '4259418', 'name': 'Indianapolis'},
   {'id': '4930956', 'name': 'Boston'}],
  'assignee_countries': [{'id': 'US', 'name': 'United States'}],
  'assignee_names': ['UNIV INDIANA RES & TECH CORP',
   'BRIGHAM & WOMENS HOSPITAL INC'],
  'assignee_state_codes': [{'id': 'US-IN', 'name': 'Indiana'},
   {'id': 'US-MA', 'name': 'Massachusetts'}],
  'assignees': [{'acronym': 'IUPUI',
    'city_name': 'Indianapolis',
    'country_code': 'US',
    'country_name': 'United States',
    'id': 'grid.257413.6',
    'latitude': 39.776966,
    'linkout': ['http://www.iupui.edu/'],
    'longitude': -86.16858,
    'name': 'Indiana University – Purdue University Indianapolis',
    'state_name': 'Indiana',
    'types': ['Education']},
   {'acronym': 'BWH',
    'city_name': 'Boston',
    'country_code': 'US',
    'country_name': 'United States',
    'id': 'grid.62560.37',
    'latitude': 42

In [49]:
#testing = {"alan d'andrea": ['ur.012727024557.25'], 'bishr omary': ['ur.014730211054.29'], 'john raymond': ['ur.0717265610.95'], 'fuad ziyadeh': ['ur.0761676730.05'], 'maria abreu': ['ur.0606477677.70'], 'michael ackerman': ['ur.01227355747.25']}

def findby_Id(nameDict):
    all_Patents= {}
    count=0
    for name, person_id in nameDict.items():
        for indv_id in person_id:
            indv_grant_list = []
            res = dsl.query_iterative(f"""search patents where researchers = "{indv_id}" return patents {search_Var}""") #[basics+categories+extras+family_count+family_id+federal_support+filing_date+granted_date+publication_ids]""")
            number = res.json['_stats']['total_count']
            print(f'{indv_id} id has {number} patents')
            listDict = res.json.get('patents', [])
            
            indv_grant_list.extend(listDict)
            
        all_Patents[name] = indv_grant_list
        count+=1
        print(f'{count}. {name}')
        print(len(all_Patents[name]))
    return all_Patents

patents = findby_Id(names)
json_filename = "all_Patents.json"

with open(json_filename, 'w') as json_file:
    json.dump(patents, json_file, indent=2)

print(f'The data has been written to {json_filename}')

Starting iteration with limit=1000 skip=0 ...
0-51 / 51 (0.99s)
===
Records extracted: 51
Starting iteration with limit=1000 skip=0 ...


ur.012727024557.25 id has 51 patents
1. alan d'andrea
51


0-22 / 22 (0.61s)
===
Records extracted: 22
Starting iteration with limit=1000 skip=0 ...


ur.01002446502.38 id has 22 patents
2. james fagin
22


0-255 / 255 (1.68s)
===
Records extracted: 255
Starting iteration with limit=1000 skip=0 ...


...

ur.01024164315.00 id has 50 patents
158. thomas wang
50


0-17 / 17 (1.10s)
===
Records extracted: 17
Starting iteration with limit=1000 skip=0 ...


ur.0652523657.55 id has 17 patents
159. myles wolf
17


0-1 / 1 (1.56s)
===
Records extracted: 1


ur.015100501062.34 id has 1 patents
160. alan yu
1
The data has been written to all_Patents.json


In [5]:
with open("all_Patents.json", 'r') as file:
    personDict = json.load(file)
    personDict.keys()

In [10]:
import csv
import json

#changes json to csv and reorganizes data
csv_filename = "dimensionsPatent_data.csv"

with open("all_Patents.json", 'r', encoding='utf-8') as file:
    personDict = json.load(file)
common_work_info_headers = ['abstract', 'additional_filters', 'application_number', 'assignee_cities', 'assignee_countries', 'assignee_names', 'assignee_state_codes', 'assignees', 'category_for', 'category_for_2020', 'category_hrcs_hc', 'category_icrp_cso', 'category_icrp_ct', 'category_rcdc', 'claims_amount', 'date', 'date_inserted', 'dimensions_url', 'family_count', 'family_id', 'filing_date', 'filing_status', 'id', 'inventor_names', 'inventors', 'ipcr', 'jurisdiction', 'kind', 'legal_status', 'original_assignee_names', 'original_assignees', 'priority_date', 'priority_year', 'publication_date', 'publication_ids', 'publication_year', 'publications', 'reference_ids', 'researchers', 'score', 'times_cited', 'title', 'year']


with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    
    header = ['Person'] + common_work_info_headers
    writer.writerow(header)

    # Write data rows
    for person, works in personDict.items():
        for work in works:  # Assuming works is a list of dictionaries now
            row = [person] + [work.get(head, '') for head in common_work_info_headers]
            writer.writerow(row)

print(f"CSV file '{csv_filename}' created successfully.")


CSV file 'dimensionsPatent_data.csv' created successfully.
